Installed PostGIS
Installed GeoPandas

//Database
createdb internshala
//POSTGIS extension
psql -d internshala -c 'create extension POSTGIS'

gr2ogr -f PostgreSQL "PG:dbname=helloworld" gadm36_IND.gpkg

psql -d internshala -c "create table tb1(State TEXT,IOCL INT,HPCL INT,BPCL INT,TOTAL INT,TOTAL2014 INT)"

psql -d internshala -c "create table tb2(State TEXT,IOCL INT,HPCL INT,BPCL INT,TOTAL INT,TOTAL2014 INT)"

//loading file 

psql -d internshala -c "\COPY tb1 FROM 'datafile.csv' delimiter ',' csv header;"

psql -d internshala -c "\COPY tb2 FROM 'datafile2.csv' delimiter ',' csv header;"

In [19]:
import geopandas

In [20]:
import psycopg2

In [21]:
con = psycopg2.connect(database="internshala")

In [22]:
sql = ( "select * from tb2;")

In [23]:
df = geopandas.read_postgis(sql,con,geom_col='state')

Error: Non-hexadecimal digit found

In [24]:
geopandas.read_file('gadm36_IND_gpkg/gadm36_IND.gpkg').head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.1_1,Nicobar Islands,None,IND.1.1.1_1,n.a. ( 2304),None,None,Taluk,Taluk,None,None,(POLYGON ((92.78778076171886 9.244166374206543...
1,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.2_1,North and Middle Andaman,None,IND.1.2.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,(POLYGON ((93.64840698242182 14.93486976623529...
2,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.3_1,South Andaman,None,IND.1.3.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,(POLYGON ((93.83969879150402 12.32081985473638...
3,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.1_1,Anantapur,None,None,Taluk,Taluk,None,None,(POLYGON ((77.35452270507812 14.52155113220221...
4,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.2_1,Dharmavaram,None,None,Taluk,Taluk,None,None,(POLYGON ((77.35340881347656 14.27067947387701...
